In [2]:
with open("defects.csv") as f:
    lines = f.readlines()
    defects = [line.strip().split(",") for line in lines[1:]]

# Defects never occur at precise integer coordinates, so a defect will never be shared by two tiles.
defect_counts = {}
for x, y in defects:
    x = int(float(x))
    if x not in defect_counts:
        defect_counts[x] = {"a": 0, "b": 0, "c": 0}

    defect_counts[x][y] += 1

print(len(defect_counts))

322


In [3]:
LENGTH = 500

BISCUITS = {
    0: {"length": 4, "value": 6, "defects": {"a": 4, "b": 2, "c": 3}, "id": 0},
    1: {"length": 8, "value": 12, "defects": {"a": 5, "b": 4, "c": 4}, "id": 1},
    2: {"length": 2, "value": 1, "defects": {"a": 1, "b": 2, "c": 1}, "id": 2},
    3: {"length": 5, "value": 8, "defects": {"a": 2, "b": 3, "c": 2}, "id": 3},
}

In [4]:
import random

# Genetic Algorithm Parameters
pop_size = 100
MUTATION_RATE = 0.01
GENERATIONS = 50

In [5]:

# Initialize a single chromosome
def initialize_chromosome():
    chromosome = [] * LENGTH
    position = 0
    while position < LENGTH:
        biscuit_type = random.choice(list(BISCUITS.keys()))
        biscuit_size = BISCUITS[biscuit_type]['length']
        if position + biscuit_size <= LENGTH:
            for i in range(biscuit_size):
                chromosome[position + i] = biscuit_type
            position += biscuit_size
        else:
            break
    return chromosome

# Initialize population
def initialize_population(pop_size):
    return [initialize_chromosome() for _ in range(pop_size)]

# Fitness function
def fitness(chromosome):
    total_value = 0
    for position, biscuit_type in enumerate(chromosome):
        if biscuit_type in [0,1,2,3]:
            biscuit = BISCUITS[biscuit_type]
            total_value += biscuit['value']
            for defect_class, max_allowed in biscuit['defects'].items():
                defect_count = sum(position <= d["position"] < position + biscuit['length'] for d in defects if d["class"] == defect_class)
                if defect_count > max_allowed:
                    total_value -= (defect_count - max_allowed) * biscuit['value'] # Penalty
    return total_value

# Selection
def select_population(population, to_keep):
    sorted_population = sorted(population, key=fitness, reverse=True)
    return sorted_population[:to_keep]

# Crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(1, LENGTH - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.choice(list(BISCUITS.keys()))
    return chromosome

# Genetic Algorithm
def genetic_algorithm(pop_size, mutation_rate, generations):
    population = initialize_population(pop_size)
    for _ in range(generations):
        new_population = []
        population = select_population(population, pop_size // 2)
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            new_population.extend([child1, child2])
        population = new_population
    best_solution = max(population, key=fitness)
    return best_solution

In [6]:
print(BISCUITS.keys())

dict_keys([0, 1, 2, 3])


In [7]:
best_chromosome = genetic_algorithm(pop_size, MUTATION_RATE, GENERATIONS)
print("Best solution:", best_chromosome)
print("Best solution fitness:", fitness(best_chromosome))

TypeError: list indices must be integers or slices, not str